## Simple Client to Access ANN prediction as an API service deployed via TF Serving
Scripts to query the flow estimation service through TF serving API


In [ ]:
import numpy as np
import json
import requests
import matplotlib.pyplot as plt

### 0. Run the TF Serving through Docker
* TF serving provides the microservice of the NN model prediction

In [ ]:
# Run following cmd in the terminal locally to start the TF serving that provides the prediction service
$ docker run -it --rm -p 8500:8500 -p 8501:8501 -v "/Users/peilunhsu/Dropbox/PEI/KTH/Courses/Master_Thesis/RISE/Githubs/D258X-Project-Traffic-Flow-Prediction/Models/NN_2featrue_temporal_1link:/models/NN_2featrue_temporal_1link" -e MODEL_NAME=NN_2featrue_temporal_1link tensorflow/serving

### 1. Load the feature array from the CSV file as a np array
* feature array shape (sample size, 7)
* 7 features: inrix_speed, inrix_travel_time, rush-hour1, off-peak1, rush-hour2, off-peak2, weekdays

In [ ]:
# load the input feature () from the csv file
X_input = np.loadtxt('features_future_test.csv', delimiter=',')

In [ ]:
# check the input feature array
print(X_input)
print(X_input.shape)
print(type(X_input[0,0]))

[[ 0.48058987 -0.41952984  0.         ...  0.          0.
   1.        ]
 [ 0.45822881 -0.41014807  0.         ...  0.          0.
   1.        ]
 [ 0.43586776 -0.4007663   0.         ...  0.          0.
   1.        ]
 ...
 [ 0.8355716  -0.60222752  0.         ...  0.          1.
   0.        ]
 [ 0.82159594 -0.59593185  0.         ...  0.          1.
   0.        ]
 [ 0.81600568 -0.59333952  0.         ...  0.          1.
   0.        ]]
(8007, 7)
<class 'numpy.float64'>


### 2. Convert the Numpy array to JSON format
* To query the estiamtion service through

In [ ]:
input_features_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_input.tolist()
})

In [ ]:
# check the json input
input_features_json

'{"signature_name": "serving_default", "instances": [[0.4805898665742332, -0.41952984441380836, 0.0, 0.0, 0.0, 0.0, 1.0], [0.4582288124275103, -0.41014807199215475, 0.0, 0.0, 0.0, 0.0, 1.0], [0.43586775828078733, -0.400766299570502, 0.0, 0.0, 0.0, 0.0, 1.0], [0.4107115723657251, -0.387804640303744, 0.0, 0.0, 0.0, 0.0, 1.0], [0.38555538645066284, -0.37484298103698593, 0.0, 0.0, 0.0, 0.0, 1.0], [0.3548089369989188, -0.36077032240450624, 0.0, 0.0, 0.0, 0.0, 1.0], [0.32965275108385655, -0.34891966250347073, 0.0, 0.0, 0.0, 0.0, 1.0], [0.3044965651687943, -0.3370690026024352, 0.0, 0.0, 0.0, 0.0, 1.0], [0.279340379253731, -0.32521834270139977, 0.0, 0.0, 0.0, 0.0, 1.0], [0.27654524748539167, -0.3239838989617088, 0.0, 0.0, 0.0, 0.0, 1.0], [0.2737501157170524, -0.32274945522201787, 0.0, 0.0, 0.0, 0.0, 1.0], [0.27095498394871204, -0.3215150114823265, 0.0, 0.0, 0.0, 0.0, 1.0], [0.427482362975778, -0.4000256333266864, 0.0, 0.0, 0.0, 0.0, 1.0], [0.4358677582807991, -0.4048399639114818, 0.0, 0.0, 0.0

### 3. Query TF Serving by sending HTTP POST request
The TF serving is hosted locally in this example <br>
(The model hosted on GCP and its client could be find in Colab files under the directory /src)

In [ ]:
SERVER_URL = 'http://localhost:8501/v1/models/NN_2featrue_temporal_1link:predict'
response = requests.post(SERVER_URL, data=input_features_json)

# raise execption in case of error
response.raise_for_status() 

response = response.json()

In [ ]:
# check the prediction/estimation results: a dictionary with key, value = 'predictions': python list containing results
print(type(response))
print(response)

<class 'dict'>
{'predictions': [[241.172058], [239.64032], [238.108673], [234.74353], [231.378433], [228.472473], [225.865204], [223.257935], [220.650665], [220.41713], [220.183548], [219.95], [239.428802], [240.887238], [246.745361], [252.679642], [258.258057], [263.758392], [266.286316], [269.011261], [270.99234], [272.973358], [275.344879], [276.492218], [278.863861], [281.235443], [282.38266], [283.529907], [281.747498], [279.747742], [277.985443], [278.683502], [278.683502], [275.37619], [272.068939], [268.761627], [269.781], [266.473694], [263.166443], [259.8591], [256.55188], [253.244614], [259.390167], [262.146942], [256.460571], [250.00589], [243.387695], [237.533844], [231.332642], [225.870453], [220.436188], [214.611298], [210.01091], [204.186], [198.361206], [193.872055], [193.440796], [193.24826], [192.511795], [189.403809], [187.030121], [188.040543], [189.050964], [190.061371], [186.696304], [187.025055], [187.353851], [187.682617], [188.011414], [188.340179], [185.21160

In [ ]:
 # convert the prediction/estimation results to numpy array for further uses, e.g., visualization, input for other model.
y_prediction = np.array(response['predictions'])
print(y_prediction[:20].round(3))

[[241.172]
 [239.64 ]
 [238.109]
 [234.744]
 [231.378]
 [228.472]
 [225.865]
 [223.258]
 [220.651]
 [220.417]
 [220.184]
 [219.95 ]
 [239.429]
 [240.887]
 [246.745]
 [252.68 ]
 [258.258]
 [263.758]
 [266.286]
 [269.011]]


The result is same as the result predicted by the original keras model when developed in Colab:  
first 20 estimation results from keras model:  
[[241.172]  
 [239.64 ]  
 [238.109]  
 [234.744]  
 [231.378]  
 [228.472]  
 [225.865]  
 [223.258]  
 [220.651]  
 [220.417]  
 [220.184]  
 [219.95 ]  
 [239.429]  
 [240.887]  
 [246.745]  
 [252.68 ]  
 [258.258]  
 [263.758]  
 [266.286]  
 [269.011]]  